In [3]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import os
import json
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer


ruta = "F:/nuevo_conocimiento/CODIGO DE TESIS (MUY IMPORTANTE)/Experimentos_con_modelos_basicos/dataset_preprocesado/dataset_abstract_preprocesado.csv"

datos = pd.read_csv(ruta)
datos

,title,abstract,categorical,categoria,cat_one_hot,abstract_limpio,abstract_preprocesado
0,Electromagnetic neutrinos in laboratory experi...,An overview of neutrino electromagnetic proper...,astrophysics,humano,"[1, 0, 0, 0, 0]",An overview of neutrino electromagnetic proper...,"['overview', 'neutrino', 'electromagnetic', 'p..."
1,Infrared complex refractive index of astrophys...,In dense and cold regions of the interstellar ...,astrophysics,humano,"[1, 0, 0, 0, 0]",In dense and cold regions of the interstellar ...,"['dense', 'cold', 'region', 'interstellar', 'm..."
2,Gamma-ray lines in modern astrophysics,Gamma-ray astronomy provides a direct window i...,astrophysics,humano,"[1, 0, 0, 0, 0]",Gamma-ray astronomy provides a direct window i...,"['gamma-ray', 'astronomy', 'provides', 'direct..."
3,From Coulomb excitation cross sections to non-...,Coulomb and nuclear dissociation of $^{17}$Ne ...,astrophysics,humano,"[1, 0, 0, 0, 0]",Coulomb and nuclear dissociation of $^{17}$Ne ...,"['coulomb', 'nuclear', 'dissociation', 'ne', '..."
4,Astrophysical Black Holes: A Compact Pedagogic...,Black holes are among the most extreme objects...,astrophysics,humano,"[1, 0, 0, 0, 0]",Black holes are among the most extreme objects...,"['black', 'hole', 'among', 'extreme', 'object'..."
...,...,...,...,...,...,...,...
7745,Superconducting LaAlO3/SrTiO3 Nanowires,Here's a rewritten version of the text using m...,superconductivity,llama3,"[0, 0, 0, 0, 1]",\n\nWe've achieved a groundbreaking breakthrou...,"[""'ve"", 'achieved', 'groundbreaking', 'breakth..."
7746,Coupling a single Nitrogen-Vacancy center to a...,Here's a rewritten version of the text in my o...,superconductivity,llama3,"[0, 0, 0, 0, 1]",\n\nWe propose a groundbreaking idea: connecti...,"['propose', 'groundbreaking', 'idea', 'connect..."
7747,Superconducting and Anti-Ferromagnetic Phases ...,Here's a rewritten version of the text using m...,superconductivity,llama3,"[0, 0, 0, 0, 1]",\n\nThis study establishes a connection betwee...,"['study', 'establishes', 'connection', 'two', ..."
7748,New Superconductivity Dome in LaFeAsO$_{1-x}$F...,Here's the rewritten text:\n\nA fascinating ph...,superconductivity,llama3,"[0, 0, 0, 0, 1]",Here's the rewritten text:\n\nA fascinating ph...,"[""'s"", 'rewritten', 'text', 'fascinating', 'ph..."


In [5]:
!pip install llama-index-embeddings-ollama

  Using cached httpx-0.27.0-py3-none-any.whl.metadata (7.2 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached greenlet-3.0.3-cp312-cp312-win_amd64.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.4 MB 2.0 MB/s eta 0:00:08
   ---------------------------------------- 0.2/15.4 MB 1.8 MB/s eta 0:00:09
    --------------------------------------- 0.3/15.4 MB 2.4 MB/s eta 0:00:07
   - -------------------------------------- 0.5/15.4 MB 3.0 MB/s eta 0:00:05
   -- ------------------------------------- 1.0/15.4 MB 4.4 MB/s eta 0:00:04
   --- ------------------------------------ 1.4/15.4 MB 5.3 MB/s eta 0:00:03
   ---- ----------------------------------- 1.8/15.4 MB 5.6 MB/s eta 0:00:03
   ---- ----------------------------------- 1.8/15.4 MB 5.8 MB/s eta 0:00:03
   ------ --------------------------------- 2.7/15.4 MB 6.6 MB/s eta 0:00:02
   ------- ----------------------------


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install llama-index

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
    --------------------------------------- 0.0/2.0 MB 2.0 MB/s eta 0:00:01
   --- ------------------------------------ 0.2/2.0 MB 1.8 MB/s eta 0:00:01
   ---- ----------------------------------- 0.2/2.0 MB 1.8 MB/s eta 0:00:01
   ----------- ---------------------------- 0.6/2.0 MB 3.0 MB/s eta 0:00:01
   ----------------- ---------------------- 0.9/2.0 MB 3.7 MB/s eta 0:00:01
   ------------------------- -------------- 1.3/2.0 MB 4.5 MB/s eta 0:00:01
   --------------------------- ------------ 1.4/2.0 MB 4.4 MB/s eta 0:00:01
   -------------------------------------- - 1.9/2.0 MB 5.8 MB/s eta 0:00:01
   ---------------------------------------  2.0/2.0 MB 5.0 MB/s eta 0:00:01
   ---------------------------------------  2.0/2.0 MB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/290.4 kB ? eta -:--:--
   ------------------- -


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="llama3",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

In [14]:
textos = datos['abstract_limpio'].tolist()
etiquetas = datos['cat_one_hot'].tolist()

pass_embedding = ollama_embedding.get_text_embedding_batch(
   textos, show_progress=True
)

print(len(pass_embedding))

Generating embeddings:   0%|          | 0/7750 [00:00<?, ?it/s]

7750


In [17]:
df = pd.DataFrame(pass_embedding)
df

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,-4.244676,0.158452,-0.522236,1.353504,2.193724,0.145882,-6.365787,-1.149827,-0.090420,-0.994674,...,-4.559592,-1.818620,0.524027,-0.807134,-0.076393,1.800469,3.273323,-0.644760,-2.981106,-2.152257
1,-3.065037,-2.881414,-2.914850,1.339224,3.417422,-0.932629,-4.233596,-0.735670,0.180235,-1.141080,...,-4.373343,-0.330858,4.040226,-1.503410,-2.323024,2.713785,4.477492,2.289947,-2.702404,-1.649539
2,-3.070732,-1.428018,-2.216700,-0.479143,1.906533,0.602597,-6.083628,0.049164,0.890967,-0.645944,...,-1.847018,-0.898792,-0.514321,0.034328,1.872394,-1.002611,3.706795,1.520496,-3.587649,-1.342454
3,-2.622239,-0.632561,-2.342142,1.255207,2.926512,1.542105,-3.420081,-0.032691,-1.680992,-0.228052,...,-3.747301,-1.629191,1.218113,-3.200677,-2.671441,0.871574,3.147774,1.651693,-3.666994,-1.957832
4,-3.394191,-2.501421,-1.145692,0.908666,2.547087,1.741781,-6.980056,-1.554243,0.560733,-1.513486,...,-4.299073,-1.310632,-0.494473,0.892572,0.887475,1.355680,5.084617,-0.403005,-2.422096,-2.066778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7745,-0.842671,-2.710984,0.475577,2.111706,3.336796,-0.762169,-1.523959,3.765510,-4.531658,-0.912678,...,-6.476543,-0.881733,3.476958,-1.006716,-1.943945,3.029183,3.099467,2.283601,-0.533691,-1.113376
7746,-2.190107,-1.985205,0.412223,0.955213,1.779113,-1.443746,-8.835557,2.578484,2.315213,-0.688466,...,-1.488121,-0.763078,-1.368813,1.556817,-0.668944,0.750065,6.157590,2.670032,-0.956725,-1.717467
7747,-4.133101,-0.364601,0.963221,0.405000,2.946863,1.416688,-4.697020,1.776428,-1.918908,-2.116655,...,-3.775192,-0.906937,-1.102986,0.378915,-1.448904,1.390008,5.989167,1.022407,-1.368397,-1.722178
7748,-2.980546,-1.887710,-0.701165,2.298929,2.832513,-1.628347,-4.829839,4.739649,-3.620148,-0.418416,...,-4.097724,-0.679182,1.937899,-0.612717,-2.014968,2.001556,4.070366,2.585566,-2.216701,-0.879591


In [18]:
df['cat_one_hot'] = etiquetas
df.to_csv('llama3_embeddings.csv', index=False)  # Para CSV